# Notebook 1 - Pobieranie i przetwarzanie danych

In [1]:
from bs4 import BeautifulSoup
import re   
import requests
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import datetime as dt
import statsmodels.api as sm

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time, os
from webdriver_manager.chrome import ChromeDriverManager

from sklearn.metrics import mean_squared_error
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.linear_model import LinearRegression, Lasso, LassoCV, Ridge, RidgeCV

%matplotlib inline

## Funkcje do scrapowania informacji o kartach

In [2]:
#pobiera nazwy kart
def get_name(soup):
    name = [link.text.strip() for link in soup.find_all('span', class_ = 'productDetailTitle')]
    return name


#pobiera z którego dodatku jest karta oraz jej rzadkość
def get_expansion_rarity_clean(soup):
    expansion_rarity_long = [link.text for link in soup.find_all('div', class_ = 'productDetailSet')]
    expansion_list = []
    rarity_list = []
    for expansion in expansion_rarity_long:
        expansion_and_rarity = expansion.strip()
        
        expansion_short = expansion_and_rarity[:-4]
        expansion_list.append(expansion_short)

        rarity = re.findall("[^()]", expansion_and_rarity[-3:])
        rarity_list.append(rarity[0])
        
    return expansion_list, rarity_list


#pobiera ceny kart
def get_price_clean(soup):
    price_long = soup.findAll("span", attrs = {"class": "stylePrice"})
    price_list = list()
    for price in price_long:
        x = price.text.strip()
        y = re.findall('\$(\d+.\d+)',x)
        conv_price = float(re.sub(",","",y[0]))
        price_list.append(conv_price)
    return price_list[::4]


#pobiera koszt zagrania karty
def get_mana_costs(soup):
    all_mana_costs_long = soup.findAll("div", attrs = {"class": "productDetailCastCost"})
    cost = []
    for card_cost in all_mana_costs_long:
        CMC_list = []
        for mana_symbol in card_cost.find_all('img'):
            CMC_list.append(re.findall('\w+', mana_symbol.get('src'))[3][-1])
        CMC_str=''
        cost.append(CMC_str.join(CMC_list))
    return cost


#wywołuje powyższe i konwertuje na ogolny koszt (powyższe daje koszt w konkretnych rodzajach many)
def get_converted_cost(soup):
    cmc = []

    for card_cost in get_mana_costs(soup):
        card_cmc=0
        for letter in card_cost:
            if letter.isdigit():
                card_cmc += int(letter)
            else: 
                card_cmc+=1
        cmc.append(card_cmc)
    return cmc


#pobiera kolor karty
def get_card_color(soup):
    color = []
    for card_cost in get_mana_costs(soup):
        card_color=[]
        for letter in card_cost:
            if letter != 'x':
                if not letter.isdigit():
                    card_color.append(letter)
                
        if card_color:
            color.append("".join(set(card_color)))
        else: color.append('colorless')
    return color

#pobiera zasady karty (ilosc znaków w tym tekście jest wykorzystywana jako jedna z cech karty)
def get_rules_text(soup):
    all_rules_text_long = soup.findAll("tr", attrs = {"class": "detailFlavortext"})
    
    rules_text = []
    for card_rules in all_rules_text_long:
        rules_text.append(card_rules.td.text.strip().replace('\n', ' '))
    
    return rules_text


#wywołuje wszystko powyżej po kolei
def all_scrape_modern():
    driver = webdriver.Chrome(ChromeDriverManager().install())
    base_url = 'https://www.cardkingdom.com/catalog/search?search=mtg_advanced&filter%5Bsort%5D=name&filter%5Bsearch%5D=mtg_advanced&filter%5Btab%5D=&filter%5Bname%5D=&filter%5Bedition%5D=&filter%5Bformat%5D=modern&filter%5Btype_mode%5D=any&filter%5Bcard_type%5D%5B10%5D=&filter%5Bpow1%5D=&filter%5Bpow2%5D=&filter%5Btuf1%5D=&filter%5Btuf2%5D=&filter%5Bconcast1%5D=&filter%5Bconcast2%5D=&filter%5Bprice_op%5D=&filter%5Bprice%5D=&filter%5Boracle_text%5D=&filter%5Bmanaprod_select%5D=any&page='
    page_number = 1
    url = base_url + str(page_number)
    number_of_pages = 400
    name = []
    expansion = []
    rarity = []
    mana_cost = []
    cmc = []
    color_identity = []
    rules_text = []
    price = []
    
    for i in range(number_of_pages):
        driver.get(url)
        soup = BeautifulSoup(driver.page_source)


        name_individual = get_name(soup)
        name.extend(name_individual)
        
        expansion_individual, rarity_individual = get_expansion_rarity_clean(soup)
        expansion.extend(expansion_individual)
        rarity.extend(rarity_individual)
        
        mana_cost_individual = get_mana_costs(soup)
        mana_cost.extend(mana_cost_individual)
        
        cmc_individual = get_converted_cost(soup)
        cmc.extend(cmc_individual)
        
        color_identity_individual = get_card_color(soup)
        color_identity.extend(color_identity_individual)
        
        rules_text_individual = get_rules_text(soup)
        rules_text.extend(rules_text_individual)
        
        price_individual = get_price_clean(soup)
        price.extend(price_individual)
        
        page_number += 1
        url = base_url + str(page_number)
        del soup
    
    return name, expansion, rarity, mana_cost, cmc, color_identity, rules_text, price


#pobiera (tym razem z wikipedii) nazwy dodatkow i daty ich wypuszczenia
def get_expansion_names_dates(soup):
    expansion_names = []
    set_dates = []

    for set_ in soup.find_all('table', class_ = 'wikitable')[1].find_all('tr')[3:115]:
        if not set_.td.get('colspan'):
            print(set_.find_all('td')[0].text)
            expansion_names.append(set_.find_all('td')[0].text.strip())

            print(set_.find_all('td')[6].text)
            set_dates.append(set_.find_all('td')[6].text)

            print('\n')
    return expansion_names, set_dates


#usuwa nawiasy z dat wypuszczenia dodatków
def remove_bracket_rel_dates(set_dates):
    set_dates_new = []
    for i in set_dates:
        if '[' in i:
            if i[-5] == '[':
                i = i[:-5]
                set_dates_new.append(i)
            else:
                i=i[:-6]
                set_dates_new.append(i)
        else: set_dates_new.append(i)        
    set_dates_new[51] = set_dates_new[51][:-5]
    return set_dates_new


#analogicznie do get_expansion_names_dates dla zestawów "Core"
def scrape_core_sets(soup):
    core_expansion_names = []
    core_set_dates = []

    for set_ in soup.find_all('table', class_ = 'wikitable')[0].find_all('tr')[2:100]:
        if not set_.td.get('colspan'):
            try: 
                if set_.find('a').text not in core_expansion_names:
                    print(set_.find('a').text)
                    core_expansion_names.append(set_.find('a').text.strip())

                print(set_.find_all('td')[4].text)
                core_set_dates.append(set_.find_all('td')[4].text)
            except: continue
    return core_expansion_names, core_set_dates


#analogicznie do remove_bracket_rel_dates
def remove_brackets_core_set_list(core_set_dates):
    core_set_dates_new = []
    for i in core_set_dates:
        if '[' in i:
            if i[-5] == '[': 
                i = i[:-5]
                core_set_dates_new.append(i)
            else:  
                i=i[:-4]
                core_set_dates_new.append(i)
        else: core_set_dates_new.append(i)        

    return core_set_dates_new


#analogicznie do get_expansion_names_dates dla uzupełniających zestawów
def scrape_reprint_comp_sets(soup):
    supp_expansion_names = []
    supp_set_dates = []


    for set_ in soup.find_all('table', class_ = 'wikitable')[4].find_all('tr')[2:100]: 
        if not set_.td.get('colspan'): 
            print(set_.find_all('td')[0].text)
            supp_expansion_names.append(set_.find_all('td')[0].text.strip())

            print(set_.find_all('td')[3].text)
            supp_set_dates.append(set_.find_all('td')[3].text.strip())

            print('\n')
    return supp_expansion_names, supp_set_dates


#analogicznie do remove_bracket_rel_dates
def remove_brackets_supp_sets(supp_set_dates):
    supp_set_dates_new = []
    for i in supp_set_dates:
        if '[' in i:
            i=i[:-5]
            supp_set_dates_new.append(i)
        else: supp_set_dates_new.append(i)        
    return supp_set_dates


#zamienia dni od wydania na date
def get_days(x):
    for i in modern_df['time_since_release']:
        try: 
            return x.days
        except: 
            return np.nan
        
def get_rarity_num(x):
    if (x == 'C'):
        return 1
    if (x == 'U'):
        return 2
    if (x == 'R'):
        return 3
    if (x == 'M'):
        return 4
        

# Scraping przy użyciu funkcji pomocniczych

In [3]:
name, expansion, rarity, mana_cost, cmc, color_identity, rules_text, price = all_scrape_modern()

C:\Users\oczos\miniconda3\envs\ml\lib\site-packages\ipykernel_launcher.py:92: DeprecationWarning: executable_path has been deprecated, please pass in a Service object


In [4]:
data_tuples_modern = list(zip(name, expansion, rarity, mana_cost, cmc, color_identity, rules_text, price))

modern_df = pd.DataFrame(data_tuples_modern, columns = ['name','expansion', 'rarity', 'cost', 'cmc', 'color_identity', 'rules_text', 'price'])

modern_df

,name,expansion,rarity,cost,cmc,color_identity,rules_text,price
0,"Ragavan, Nimble Pilferer",Modern Horizons 2,M,r,1,r,"Whenever Ragavan, Nimble Pilferer deals combat...",79.99
1,"Sheoldred, the Apocalypse",Dominaria United,M,2bb,4,b,"Deathtouch Whenever you draw a card, you gain ...",79.99
2,Urza's Saga,Modern Horizons 2,R,,0,colorless,"(As this Saga enters and after your draw step,...",42.99
3,Fable of the Mirror-Breaker,Kamigawa: Neon Dynasty,R,2r,3,r,"(As this Saga enters and after your draw step,...",29.99
4,Polluted Delta,Khans of Tarkir,R,,0,colorless,", Pay 1 life, Sacrifice Polluted Delta: Search...",44.99
...,...,...,...,...,...,...,...,...
9995,Clockspinning,Time Spiral,C,u,1,u,Buyback (You may pay an additional as you ca...,0.79
9996,Explosive Singularity (Showcase),Kamigawa: Neon Dynasty Variants,M,8rr,10,r,"As an additional cost to cast this spell, you ...",0.79
9997,Drizzt Do'Urden (Showcase),Adventures in the Forgotten Realms Variants,R,3gw,5,gw,Double strike When Drizzt Do'Urden enters the ...,0.79
9998,"Gargos, Vicious Watcher",Core Set 2020,R,3ggg,6,g,Vigilance Hydra spells you cast cost less to ...,0.79


In [5]:
modern_df.shape

(10000, 8)

In [6]:
#backup
filename = 'modern_data_baseline_PICKLE'
outfile = open(filename,'wb')
pickle.dump(modern_df,outfile)
outfile.close()

### Usuwam specjalne rzadkości i podstawowe karty krain

In [7]:
modern_df = pd.read_pickle('modern_data_baseline_PICKLE')

mask = (modern_df['rarity'] != 'L') & (modern_df['rarity'] != 'S')

modern_df = modern_df[mask]

modern_df.shape

(9373, 8)

### Usuwam Extended Art, Borderless, Showcase, Godzilla Series

In [8]:
ex_art_mask = ~modern_df['name'].str.contains("Extended Art")

modern_df = modern_df[ex_art_mask] 

borderless_mask = ~modern_df['name'].str.contains("Borderless")

modern_df = modern_df[borderless_mask] 

showcase_mask = ~modern_df['name'].str.contains("Showcase")

modern_df = modern_df[showcase_mask]

godzilla_mask = ~modern_df['name'].str.contains("Godzilla")

modern_df = modern_df[godzilla_mask]

modern_df.shape

(8618, 8)

### Zmieniam indeks na kolumnę z nazwą

In [9]:
modern_df.set_index('name', drop=True, inplace = True)


modern_df.head()

,expansion,rarity,cost,cmc,color_identity,rules_text,price
name,,,,,,,
"Ragavan, Nimble Pilferer",Modern Horizons 2,M,r,1,r,"Whenever Ragavan, Nimble Pilferer deals combat...",79.99
"Sheoldred, the Apocalypse",Dominaria United,M,2bb,4,b,"Deathtouch Whenever you draw a card, you gain ...",79.99
Urza's Saga,Modern Horizons 2,R,,0,colorless,"(As this Saga enters and after your draw step,...",42.99
Fable of the Mirror-Breaker,Kamigawa: Neon Dynasty,R,2r,3,r,"(As this Saga enters and after your draw step,...",29.99
Polluted Delta,Khans of Tarkir,R,,0,colorless,", Pay 1 life, Sacrifice Polluted Delta: Search...",44.99


In [10]:
#fix do color identity
modern_df['color_identity'].replace('n', 'colorless', inplace=True)
modern_df['color_identity'].replace('e', 'colorless', inplace=True)

modern_df.head()

,expansion,rarity,cost,cmc,color_identity,rules_text,price
name,,,,,,,
"Ragavan, Nimble Pilferer",Modern Horizons 2,M,r,1,r,"Whenever Ragavan, Nimble Pilferer deals combat...",79.99
"Sheoldred, the Apocalypse",Dominaria United,M,2bb,4,b,"Deathtouch Whenever you draw a card, you gain ...",79.99
Urza's Saga,Modern Horizons 2,R,,0,colorless,"(As this Saga enters and after your draw step,...",42.99
Fable of the Mirror-Breaker,Kamigawa: Neon Dynasty,R,2r,3,r,"(As this Saga enters and after your draw step,...",29.99
Polluted Delta,Khans of Tarkir,R,,0,colorless,", Pay 1 life, Sacrifice Polluted Delta: Search...",44.99


## Kodowanie koloru do 5 kolumn

In [11]:
is_w = []
is_u = []
is_b = []
is_r = []
is_g = []

for card_color_identity in modern_df['color_identity']:
    if card_color_identity != 'colorless':
        if 'w' in card_color_identity:
            is_w.append(1)
        else: 
            is_w.append(0)

        if 'u' in card_color_identity:
            is_u.append(1)
        else: 
            is_u.append(0)

        if 'b' in card_color_identity:
            is_b.append(1)
        else: 
            is_b.append(0)

        if 'r' in card_color_identity:
            is_r.append(1)
        else: 
            is_r.append(0)

        if 'g' in card_color_identity:
            is_g.append(1)
        else: 
            is_g.append(0)
    else: 
        is_w.append(0)
        is_u.append(0)
        is_b.append(0)
        is_r.append(0)
        is_g.append(0)
        

data_tuples = list(zip(is_w, is_u, is_b, is_r, is_g))
color_var_df = pd.DataFrame(data_tuples, columns = ['is_w', 'is_u', 'is_b', 'is_r', 'is_g'], index=modern_df.index)

color_var_df.head()

,is_w,is_u,is_b,is_r,is_g
name,,,,,
"Ragavan, Nimble Pilferer",0,0,0,1,0
"Sheoldred, the Apocalypse",0,0,1,0,0
Urza's Saga,0,0,0,0,0
Fable of the Mirror-Breaker,0,0,0,1,0
Polluted Delta,0,0,0,0,0


In [12]:
modern_df = pd.concat([modern_df, color_var_df], axis=1, sort=False)

modern_df.head()

,expansion,rarity,cost,cmc,color_identity,rules_text,price,is_w,is_u,is_b,is_r,is_g
name,,,,,,,,,,,,
"Ragavan, Nimble Pilferer",Modern Horizons 2,M,r,1,r,"Whenever Ragavan, Nimble Pilferer deals combat...",79.99,0,0,0,1,0
"Sheoldred, the Apocalypse",Dominaria United,M,2bb,4,b,"Deathtouch Whenever you draw a card, you gain ...",79.99,0,0,1,0,0
Urza's Saga,Modern Horizons 2,R,,0,colorless,"(As this Saga enters and after your draw step,...",42.99,0,0,0,0,0
Fable of the Mirror-Breaker,Kamigawa: Neon Dynasty,R,2r,3,r,"(As this Saga enters and after your draw step,...",29.99,0,0,0,1,0
Polluted Delta,Khans of Tarkir,R,,0,colorless,", Pay 1 life, Sacrifice Polluted Delta: Search...",44.99,0,0,0,0,0


### Koduję rzadkość karty w formie liczby 1-4

In [13]:
modern_df['rarity_num'] = modern_df.rarity.apply(lambda x: get_rarity_num(x))
modern_df.head()

,expansion,rarity,cost,cmc,color_identity,rules_text,price,is_w,is_u,is_b,is_r,is_g,rarity_num
name,,,,,,,,,,,,,
"Ragavan, Nimble Pilferer",Modern Horizons 2,M,r,1,r,"Whenever Ragavan, Nimble Pilferer deals combat...",79.99,0,0,0,1,0,4
"Sheoldred, the Apocalypse",Dominaria United,M,2bb,4,b,"Deathtouch Whenever you draw a card, you gain ...",79.99,0,0,1,0,0,4
Urza's Saga,Modern Horizons 2,R,,0,colorless,"(As this Saga enters and after your draw step,...",42.99,0,0,0,0,0,3
Fable of the Mirror-Breaker,Kamigawa: Neon Dynasty,R,2r,3,r,"(As this Saga enters and after your draw step,...",29.99,0,0,0,1,0,3
Polluted Delta,Khans of Tarkir,R,,0,colorless,", Pay 1 life, Sacrifice Polluted Delta: Search...",44.99,0,0,0,0,0,3


### Dodaję ilość znaków w instrukcji karty 

In [14]:
rules_text_len_mod = []
for i in modern_df['rules_text']:
    rules_text_len_mod.append(len(i))
    
modern_df['rules_text_len'] = rules_text_len_mod
modern_df.head()

,expansion,rarity,cost,cmc,color_identity,rules_text,price,is_w,is_u,is_b,is_r,is_g,rarity_num,rules_text_len
name,,,,,,,,,,,,,,
"Ragavan, Nimble Pilferer",Modern Horizons 2,M,r,1,r,"Whenever Ragavan, Nimble Pilferer deals combat...",79.99,0,0,0,1,0,4,358
"Sheoldred, the Apocalypse",Dominaria United,M,2bb,4,b,"Deathtouch Whenever you draw a card, you gain ...",79.99,0,0,1,0,0,4,106
Urza's Saga,Modern Horizons 2,R,,0,colorless,"(As this Saga enters and after your draw step,...",42.99,0,0,0,0,0,3,381
Fable of the Mirror-Breaker,Kamigawa: Neon Dynasty,R,2r,3,r,"(As this Saga enters and after your draw step,...",29.99,0,0,0,1,0,3,568
Polluted Delta,Khans of Tarkir,R,,0,colorless,", Pay 1 life, Sacrifice Polluted Delta: Search...",44.99,0,0,0,0,0,3,136


In [15]:
#backup
filename = 'modern_data_rarity_color_rules_len_coded_PICKLE'
outfile = open(filename,'wb')
pickle.dump(modern_df,outfile)
outfile.close()

In [16]:
modern_df = pd.read_pickle('modern_data_rarity_color_rules_len_coded_PICKLE')

### Dodaję ilość wydań karty

In [17]:
num_printings_dict = modern_df.index.value_counts().to_dict()
modern_df['num_printings']= modern_df.index.map(num_printings_dict)

modern_df.head()

,expansion,rarity,cost,cmc,color_identity,rules_text,price,is_w,is_u,is_b,is_r,is_g,rarity_num,rules_text_len,num_printings
name,,,,,,,,,,,,,,,
"Ragavan, Nimble Pilferer",Modern Horizons 2,M,r,1,r,"Whenever Ragavan, Nimble Pilferer deals combat...",79.99,0,0,0,1,0,4,358,1
"Sheoldred, the Apocalypse",Dominaria United,M,2bb,4,b,"Deathtouch Whenever you draw a card, you gain ...",79.99,0,0,1,0,0,4,106,1
Urza's Saga,Modern Horizons 2,R,,0,colorless,"(As this Saga enters and after your draw step,...",42.99,0,0,0,0,0,3,381,1
Fable of the Mirror-Breaker,Kamigawa: Neon Dynasty,R,2r,3,r,"(As this Saga enters and after your draw step,...",29.99,0,0,0,1,0,3,568,1
Polluted Delta,Khans of Tarkir,R,,0,colorless,", Pay 1 life, Sacrifice Polluted Delta: Search...",44.99,0,0,0,0,0,3,136,3


### Dodaję czas od wydania

In [18]:
mtg_set_wiki_url = 'https://en.wikipedia.org/wiki/List_of_Magic:_The_Gathering_sets'

response = requests.get(mtg_set_wiki_url)

response.status_code  

200

In [19]:
page = response.text
soup = BeautifulSoup(page)

### Scrapuję nazwy dodatków i daty ich wydania z Wikipedii

In [20]:
expansion_names, set_dates = get_expansion_names_dates(soup)

Arabian Nights

December 1993[31]



Antiquities

March 1994[32]



Legends

June 1994[33]



The Dark

August 1994[34]



Fallen Empires

November 1994[35]



Ice Age

June 1995[36]



Homelands[IX]

October 1995[37]



Alliances

June 10, 1996[10]



Mirage

October 7, 1996[10]



Visions

February 3, 1997[41]



Weatherlight

June 9, 1997[10]



Tempest

October 13, 1997[10]



Stronghold

March 2, 1998[10]



Exodus

June 15, 1998[10]



Urza's Saga

October 12, 1998[47]



Urza's Legacy

February 15, 1999[10]



Urza's Destiny

June 7, 1999[49]



Mercadian Masques

October 4, 1999[51]



Nemesis

February 14, 2000[53]



Prophecy

June 5, 2000[54]



Invasion

October 2, 2000[55]



Planeshift

February 5, 2001[56]



Apocalypse

June 4, 2001



Odyssey

October 1, 2001



Torment

February 4, 2002[58]



Judgment

May 27, 2002[59]



Onslaught

October 7, 2002[60]



Legions

February 3, 2003



Scourge

May 26, 2003[62]



Mirrodin

October 3, 2003[63]



Darksteel

February 6,

In [21]:
print(len(expansion_names), len(set_dates))

85 85


In [22]:
# usuwam [] z dat

set_dates_new = remove_bracket_rel_dates(set_dates)
len(set_dates_new)

85

### Tworzę df z danymi z Wikipedii

In [23]:
sets_and_rel_dates = pd.DataFrame(list(zip(expansion_names, set_dates_new)), columns = ['expansion', 'rel_date'])

sets_and_rel_dates.head()

,expansion,rel_date
0,Arabian Nights,December 1993
1,Antiquities,March 1994
2,Legends,June 1994
3,The Dark,August 1994
4,Fallen Empires,November 1994


###  Analogicznie dla core setów

In [24]:
core_expansion_names, core_set_dates = scrape_core_sets(soup)

Limited Edition Alpha
August 5, 1993[6]

Limited Edition Beta
October 1993[7]

Unlimited Edition
December 1993[6]

Revised Edition
April 1994[8]

Fourth Edition
April 1995[9]

Fifth Edition
March 24, 1997[10]

Classic Sixth Edition
April 28, 1999[10]

Seventh Edition
April 11, 2001[10]

Eighth Edition
July 28, 2003[11]

Ninth Edition
July 29, 2005[12]

Tenth Edition
July 13, 2007[13]

Magic 2010
July 17, 2009[15]

Magic 2011
July 16, 2010[16]

Magic 2012
July 15, 2011[17]

Magic 2013
July 13, 2012[18]

Magic 2014
July 19, 2013[19]

Magic 2015
July 18, 2014[20]

Magic Origins
July 17, 2015[22]

Core Set 2019
July 13, 2018[23]

Core Set 2020
July 12, 2019[24]

Core Set 2021
July 3, 2020[25]



In [25]:
core_set_dates_new = remove_brackets_core_set_list(core_set_dates)
core_sets_and_rel_dates = pd.DataFrame(list(zip(core_expansion_names, core_set_dates_new)), columns = ['expansion', 'rel_date'])
core_sets_and_rel_dates.head()

,expansion,rel_date
0,Limited Edition Alpha,"August 5, 1993"
1,Limited Edition Beta,October 1993
2,Unlimited Edition,December 1993
3,Revised Edition,April 1994
4,Fourth Edition,April 1995


### Analogicznie dla zestawów kompilacyjnych i z reprintami

In [26]:
supp_expansion_names, supp_set_dates = scrape_reprint_comp_sets(soup)

Chronicles[XVII]

July 1995[163]



Rivals Quick Start Set

July 1996[164]



Multiverse Gift Box

November 1996[165]



Anthologies

November 1998[166]



Battle Royale Box Set

November 12, 1999[167]



Beatdown Box Set

December 2000[168]



Deckmasters: Garfield vs. Finkel

September 17, 2001[169]



Premium Foil Booster

January 8, 2010[170]



Duels of the Planeswalkers (decks)

June 4, 2010[171]



Modern Event Deck

May 30, 2014[172]



Mystery  Booster

March 13, 2020[173]



Time Spiral Remastered

March 19, 2021



Dominaria Remastered

January 13, 2023



Renaissance (French/German)

August 1995[163]



Rinascimento (Italian)

August 1995[163]



Duel Decks: Elves vs. Goblins

November 16, 2007[174]



Duel Decks: Jace vs. Chandra

November 7, 2008[175]



Duel Decks: Divine vs. Demonic

April 10, 2009[176]



Duel Decks: Garruk vs. Liliana

October 30, 2009[177]



Duel Decks: Phyrexia vs. the Coalition

March 19, 2010[178]



Duel Decks: Elspeth vs. Tezzeret

September 3,

In [27]:
supp_set_dates_new = remove_brackets_supp_sets(supp_set_dates)
supp_sets_and_rel_dates = pd.DataFrame(list(zip(supp_expansion_names, supp_set_dates_new)), columns = ['expansion', 'rel_date'])
supp_sets_and_rel_dates.head()

,expansion,rel_date
0,Chronicles[XVII],July 1995[163]
1,Rivals Quick Start Set,July 1996[164]
2,Multiverse Gift Box,November 1996[165]
3,Anthologies,November 1998[166]
4,Battle Royale Box Set,"November 12, 1999[167]"


In [28]:
sets_and_rel_dates = sets_and_rel_dates.append(core_sets_and_rel_dates, ignore_index = True)
sets_and_rel_dates = sets_and_rel_dates.append(supp_sets_and_rel_dates, ignore_index = True)

sets_and_rel_dates.tail()

,expansion,rel_date
183,2014 Holiday Gift Box,"November 14, 2014[225]"
184,2015 Holiday Gift Box,"November 6, 2015[226]"
185,Shadows over Innistrad: The Gift Box,"May 13, 2016[227]"
186,Kaladesh: The Gift Box,N/A
187,Throne of Eldraine Bundle Gift Edition [228],"November 15, 2019"


In [29]:
sets_and_rel_dates = sets_and_rel_dates.append({'expansion' : 'Modern Horizons' , 'rel_date' : 'June 14, 2019'} , ignore_index=True)

In [30]:
sets_and_rel_dates['expansion'][65:90]

65                Fate Reforged
66            Dragons of Tarkir
67          Battle for Zendikar
68        Oath of the Gatewatch
69       Shadows over Innistrad
70                Eldritch Moon
71                     Kaladesh
72                Aether Revolt
73                     Amonkhet
74          Hour of Devastation
75                       Ixalan
76             Rivals of Ixalan
77                    Dominaria
78            Guilds of Ravnica
79           Ravnica Allegiance
80        War of the Spark[132]
81           Throne of Eldraine
82          Theros Beyond Death
83    Ikoria: Lair of Behemoths
84              Zendikar Rising
85        Limited Edition Alpha
86         Limited Edition Beta
87            Unlimited Edition
88              Revised Edition
89               Fourth Edition
Name: expansion, dtype: object

### Manualne zmiany w formatowaniu

In [31]:
good = ['2010 Core Set', '2011 Core Set','2012 Core Set', '2013 Core Set', '2014 Core Set', '2015 Core Set']
bad = ['Magic 2010', 'Magic 2011', 'Magic 2012', 'Magic 2013', 'Magic 2014', 'Magic 2015',]

for i, j in zip(bad, good):
    sets_and_rel_dates= sets_and_rel_dates.replace(i,j)
    
bad2= ['Shadows over Innistrad','Modern Masters 2015 Edition','Fourth Edition','Fifth Edition','Classic Sixth Edition','Seventh Edition', 'Eighth Edition','Ninth Edition', 'Tenth Edition', 'Ravnica: City of Guilds', 'Coldsnap[IX]']

good2=['Shadows Over Innistrad','Modern Masters 2015','4th Edition','5th Edition','6th Edition','7th Edition','8th Edition','9th Edition','10th Edition', 'Ravnica', 'Coldsnap']

for i, j in zip(bad2, good2):
    sets_and_rel_dates= sets_and_rel_dates.replace(i,j)
    
good3 = ['War of the Spark', '3rd Edition','Theros: Beyond Death', 'Alpha', 'Beta', 'Unlimited']
bad3 = ['War of the Spark[132]','Revised Edition','Theros Beyond Death','Limited Edition Alpha','Limited Edition Beta','Unlimited Edition']

for i, j in zip(bad3, good3):
    sets_and_rel_dates= sets_and_rel_dates.replace(i,j)

Masa manualnego dodawania setów przez różnice między nazwami na card kingdom i wikipedii

In [32]:
sets_and_rel_dates.loc[len(sets_and_rel_dates.index)] = ['Modern Horizons 2','June 18, 2021']
sets_and_rel_dates.loc[len(sets_and_rel_dates.index)] = ['Dominaria United','September 09, 2022']
sets_and_rel_dates.loc[len(sets_and_rel_dates.index)] = ['Kamigawa: Neon Dynasty','February 18, 2022']
sets_and_rel_dates.loc[len(sets_and_rel_dates.index)] = ['Phyrexia: All Will Be One','February 10, 2023']
sets_and_rel_dates.loc[len(sets_and_rel_dates.index)] = ['Commander Legends','November 20, 2020']
sets_and_rel_dates.loc[len(sets_and_rel_dates.index)] = ['Mystery Booster/The List','March 13, 2020']
sets_and_rel_dates.loc[len(sets_and_rel_dates.index)] = ['Streets of New Capenna','April 29, 2022']
sets_and_rel_dates.loc[len(sets_and_rel_dates.index)] = ['Commander Legends: Battle for Baldur\'s Gate','June 10, 2022']
sets_and_rel_dates.loc[len(sets_and_rel_dates.index)] = ['Theros Beyond Death','January 17, 2020']
sets_and_rel_dates.loc[len(sets_and_rel_dates.index)] = ['Adventures in the Forgotten Realms Commander Decks','July 23, 2021']
sets_and_rel_dates.loc[len(sets_and_rel_dates.index)] = ['Adventures in the Forgotten Realms','July 23, 2021']
sets_and_rel_dates.loc[len(sets_and_rel_dates.index)] = ['Battlebond','June 8, 2018']
sets_and_rel_dates.loc[len(sets_and_rel_dates.index)] = ['Collectors Ed','December 01, 1993']
sets_and_rel_dates.loc[len(sets_and_rel_dates.index)] = ['Collectors Ed Intl','December 01, 1993']
sets_and_rel_dates.loc[len(sets_and_rel_dates.index)] = ['Commander','June 17, 2011']
sets_and_rel_dates.loc[len(sets_and_rel_dates.index)] = ['Commander 2013','November 1, 2013']
sets_and_rel_dates.loc[len(sets_and_rel_dates.index)] = ['Commander 2014','November 7, 2014']
sets_and_rel_dates.loc[len(sets_and_rel_dates.index)] = ['Commander 2015','November 13, 2015']
sets_and_rel_dates.loc[len(sets_and_rel_dates.index)] = ['Commander 2016','November 11, 2016']
sets_and_rel_dates.loc[len(sets_and_rel_dates.index)] = ['Commander 2017','August 25, 2017']
sets_and_rel_dates.loc[len(sets_and_rel_dates.index)] = ['Commander 2018','August 10, 2018']
sets_and_rel_dates.loc[len(sets_and_rel_dates.index)] = ['Commander 2019','August 23, 2019']
sets_and_rel_dates.loc[len(sets_and_rel_dates.index)] = ['Commander 2020','May 15, 2020']
sets_and_rel_dates.loc[len(sets_and_rel_dates.index)] = ['Commander 2021','April 23, 2021']
sets_and_rel_dates.loc[len(sets_and_rel_dates.index)] = ['Commander\'s Arsenal','November 2, 2012']
sets_and_rel_dates.loc[len(sets_and_rel_dates.index)] = ['Commander Anthology Vol. II','June 8, 2018']
sets_and_rel_dates.loc[len(sets_and_rel_dates.index)] = ['Commander Anthology','June 9, 2017']
sets_and_rel_dates.loc[len(sets_and_rel_dates.index)] = ['Commander Legends Variants','November 20, 2020']
sets_and_rel_dates.loc[len(sets_and_rel_dates.index)] = ['Conspiracy - Take the Crown','August 26, 2016']
sets_and_rel_dates.loc[len(sets_and_rel_dates.index)] = ['Conspiracy','June 6, 2014']
sets_and_rel_dates.loc[len(sets_and_rel_dates.index)] = ['Dominaria Remastered Variants','January 13, 2023']
sets_and_rel_dates.loc[len(sets_and_rel_dates.index)] = ['Dominaria United Commander Decks','September 09, 2022']
sets_and_rel_dates.loc[len(sets_and_rel_dates.index)] = ['Double Masters','August 7, 2020']
sets_and_rel_dates.loc[len(sets_and_rel_dates.index)] = ['Double Masters 2022','July 8, 2022']
sets_and_rel_dates.loc[len(sets_and_rel_dates.index)] = ['Double Masters 2022 Variants','July 8, 2022']
sets_and_rel_dates.loc[len(sets_and_rel_dates.index)] = ['Double Masters Box Toppers','August 7, 2020']
sets_and_rel_dates.loc[len(sets_and_rel_dates.index)] = ['Eternal Masters','June 10, 2016']
sets_and_rel_dates.loc[len(sets_and_rel_dates.index)] = ['Iconic Masters','November 17, 2017']
sets_and_rel_dates.loc[len(sets_and_rel_dates.index)] = ['Innistrad: Crimson Vow','November 19, 2021']
sets_and_rel_dates.loc[len(sets_and_rel_dates.index)] = ['Innistrad: Crimson Vow Commander Decks ','November 19, 2021']
sets_and_rel_dates.loc[len(sets_and_rel_dates.index)] = ['Innistrad: Double Feature','January 28, 2022']
sets_and_rel_dates.loc[len(sets_and_rel_dates.index)] = ['Innistrad: Midnight Hunt','September 24, 2021']
sets_and_rel_dates.loc[len(sets_and_rel_dates.index)] = ['Innistrad: Midnight Hunt Commander Decks','September 24, 2021']
sets_and_rel_dates.loc[len(sets_and_rel_dates.index)] = ['Innistrad: Midnight Hunt Variants','September 24, 2021']
sets_and_rel_dates.loc[len(sets_and_rel_dates.index)] = ['Jumpstart','July 17, 2020']
sets_and_rel_dates.loc[len(sets_and_rel_dates.index)] = ['Jumpstart 2022','December 2, 2022']
sets_and_rel_dates.loc[len(sets_and_rel_dates.index)] = ['Kaldheim','February 5, 2021']
sets_and_rel_dates.loc[len(sets_and_rel_dates.index)] = ['Kaldheim Commander Decks','February 2, 2021']
sets_and_rel_dates.loc[len(sets_and_rel_dates.index)] = ['Kamigawa: Neon Dynasty Commander Decks','February 18, 2022']
sets_and_rel_dates.loc[len(sets_and_rel_dates.index)] = ['Masterpiece Series: Mythic Edition','October 03, 2018']
sets_and_rel_dates.loc[len(sets_and_rel_dates.index)] = ['Masters 25','March 16, 2018']
sets_and_rel_dates.loc[len(sets_and_rel_dates.index)] = ['Modern Horizons - Retro Frames','June 14, 2019']
sets_and_rel_dates.loc[len(sets_and_rel_dates.index)] = ['Modern Horizons 2 Variants','June 18, 2021']
sets_and_rel_dates.loc[len(sets_and_rel_dates.index)] = ['Modern Masters','June 7, 2013']
sets_and_rel_dates.loc[len(sets_and_rel_dates.index)] = ['Modern Masters 2015','May 22, 2015']
sets_and_rel_dates.loc[len(sets_and_rel_dates.index)] = ['Modern Masters 2017','March 17, 2017']
sets_and_rel_dates.loc[len(sets_and_rel_dates.index)] = ['Portal','June 01, 1997']
sets_and_rel_dates.loc[len(sets_and_rel_dates.index)] = ['Portal 3K','May 01, 1999']
sets_and_rel_dates.loc[len(sets_and_rel_dates.index)] = ['Portal II','June 01, 1998']
sets_and_rel_dates.loc[len(sets_and_rel_dates.index)] = ['Promo Pack','January 01, 2000']
sets_and_rel_dates.loc[len(sets_and_rel_dates.index)] = ['Promotional','January 01, 2000']
sets_and_rel_dates.loc[len(sets_and_rel_dates.index)] = ['Secret Lair','January 01, 2020']
sets_and_rel_dates.loc[len(sets_and_rel_dates.index)] = ['Starter Commander Decks','July 01, 2000']
sets_and_rel_dates.loc[len(sets_and_rel_dates.index)] = ['Streets of New Capenna Commander Decks','April 29, 2022']
sets_and_rel_dates.loc[len(sets_and_rel_dates.index)] = ['Strixhaven Mystical Archive','April 23, 2021']
sets_and_rel_dates.loc[len(sets_and_rel_dates.index)] = ['Strixhaven Mystical Archive JPN','April 23, 2021']
sets_and_rel_dates.loc[len(sets_and_rel_dates.index)] = ['Strixhaven: School of Mages','April 23, 2021']
sets_and_rel_dates.loc[len(sets_and_rel_dates.index)] = ['The Brothers\' War','November 18, 2022']
sets_and_rel_dates.loc[len(sets_and_rel_dates.index)] = ['The Brothers\' War Commander Decks','November 18, 2022']
sets_and_rel_dates.loc[len(sets_and_rel_dates.index)] = ['The Brothers\' War Retro Artifacts','November 18, 2022']
sets_and_rel_dates.loc[len(sets_and_rel_dates.index)] = ['Ultimate Masters','December 7, 2018']
sets_and_rel_dates.loc[len(sets_and_rel_dates.index)] = ['Universes Beyond: Warhammer 40,000','October 7, 2022']
sets_and_rel_dates.loc[len(sets_and_rel_dates.index)] = ['Universes Beyond: Transformers','November 18, 2022']
sets_and_rel_dates.loc[len(sets_and_rel_dates.index)] = ['World Championships','January 01, 1998']
sets_and_rel_dates.loc[len(sets_and_rel_dates.index)] = ['Zendikar Rising Commander Decks','September 25, 2020']
sets_and_rel_dates.loc[len(sets_and_rel_dates.index)] = ['Zendikar Rising Expeditions','September 25, 2020']

In [33]:
sets_and_rel_dates.loc[sets_and_rel_dates['expansion']=='Rise of the Eldrazi', ['rel_date']]='April 23, 2010'
sets_and_rel_dates.loc[sets_and_rel_dates['expansion']=='From the Vault: Relics', ['rel_date']]='August 27, 2010'

In [34]:
sets_and_rel_dates['rel_date'].replace('N/A', np.nan)

0           December 1993
1              March 1994
2               June 1994
3             August 1994
4           November 1994
              ...        
260       October 7, 2022
261     November 18, 2022
262      January 01, 1998
263    September 25, 2020
264    September 25, 2020
Name: rel_date, Length: 265, dtype: object

In [35]:
sets_and_rel_dates['rel_date'] = sets_and_rel_dates['rel_date'].apply(lambda x:x.strip())

sets_and_rel_dates['rel_date'][22]

'June 4, 2001'

In [36]:
sets_and_rel_dates['rel_date'][54] = sets_and_rel_dates['rel_date'][54].replace('[98]', '')

### Formatowanie dat

In [37]:
dates_format_correct = []
for date_ in sets_and_rel_dates['rel_date']:
    try:
        dates_format_correct.append(pd.to_datetime(date_))
    except: dates_format_correct.append(np.nan)

dates_format_correct

[Timestamp('1993-12-01 00:00:00'),
 Timestamp('1994-03-01 00:00:00'),
 Timestamp('1994-06-01 00:00:00'),
 Timestamp('1994-08-01 00:00:00'),
 Timestamp('1994-11-01 00:00:00'),
 Timestamp('1995-06-01 00:00:00'),
 Timestamp('1995-10-01 00:00:00'),
 Timestamp('1996-06-10 00:00:00'),
 Timestamp('1996-10-07 00:00:00'),
 Timestamp('1997-02-03 00:00:00'),
 Timestamp('1997-06-09 00:00:00'),
 Timestamp('1997-10-13 00:00:00'),
 Timestamp('1998-03-02 00:00:00'),
 Timestamp('1998-06-15 00:00:00'),
 Timestamp('1998-10-12 00:00:00'),
 Timestamp('1999-02-15 00:00:00'),
 Timestamp('1999-06-07 00:00:00'),
 Timestamp('1999-10-04 00:00:00'),
 Timestamp('2000-02-14 00:00:00'),
 Timestamp('2000-06-05 00:00:00'),
 Timestamp('2000-10-02 00:00:00'),
 Timestamp('2001-02-05 00:00:00'),
 Timestamp('2001-06-04 00:00:00'),
 Timestamp('2001-10-01 00:00:00'),
 Timestamp('2002-02-04 00:00:00'),
 Timestamp('2002-05-27 00:00:00'),
 Timestamp('2002-10-07 00:00:00'),
 Timestamp('2003-02-03 00:00:00'),
 Timestamp('2003-05-

In [38]:
sets_and_rel_dates['rel_date_correct'] = dates_format_correct

In [39]:
expansion_date_dict = pd.Series(sets_and_rel_dates['rel_date_correct'].values,index=sets_and_rel_dates['expansion']).to_dict()
modern_df['rel_date'] = modern_df['expansion'].map(expansion_date_dict)
modern_df.head()

,expansion,rarity,cost,cmc,color_identity,rules_text,price,is_w,is_u,is_b,is_r,is_g,rarity_num,rules_text_len,num_printings,rel_date
name,,,,,,,,,,,,,,,,
"Ragavan, Nimble Pilferer",Modern Horizons 2,M,r,1,r,"Whenever Ragavan, Nimble Pilferer deals combat...",79.99,0,0,0,1,0,4,358,1,2021-06-18
"Sheoldred, the Apocalypse",Dominaria United,M,2bb,4,b,"Deathtouch Whenever you draw a card, you gain ...",79.99,0,0,1,0,0,4,106,1,2022-09-09
Urza's Saga,Modern Horizons 2,R,,0,colorless,"(As this Saga enters and after your draw step,...",42.99,0,0,0,0,0,3,381,1,2021-06-18
Fable of the Mirror-Breaker,Kamigawa: Neon Dynasty,R,2r,3,r,"(As this Saga enters and after your draw step,...",29.99,0,0,0,1,0,3,568,1,2022-02-18
Polluted Delta,Khans of Tarkir,R,,0,colorless,", Pay 1 life, Sacrifice Polluted Delta: Search...",44.99,0,0,0,0,0,3,136,3,2014-09-26


In [40]:
mask = modern_df["rel_date"].isna()
df = modern_df[mask]
df = df.drop_duplicates('expansion')
df

,expansion,rarity,cost,cmc,color_identity,rules_text,price,is_w,is_u,is_b,is_r,is_g,rarity_num,rules_text_len,num_printings,rel_date
name,,,,,,,,,,,,,,,,
"Sheoldred, the Apocalypse (Phyrexian)",Dominaria United Variants,M,2bb,4,b,"Deathtouch Whenever you draw a card, you gain ...",74.99,0,0,1,0,0,4,106,1,NaT
"Liliana, Dreadhorde General",War of the Spark\n,M,4bb,6,b,"Whenever a creature you control dies, draw a c...",27.99,0,0,1,0,0,4,247,1,NaT
"Rin and Seri, Inseparable (Buy-a-Box Foil)",Promotional\n,M,1rgw,4,rgw,"Whenever you cast a Dog spell, create a 1/1 gr...",20.99,1,0,0,1,1,4,292,1,NaT
"Vorinclex, Monstrous Raider (Phyrexian)",Kaldheim Variants,M,4gg,6,g,"Trample, haste If you would put one or more co...",49.99,0,0,0,0,1,4,352,1,NaT
Altar of the Brood,Khans of Tarkir\n,R,1,1,colorless,Whenever another permanent enters the battlefi...,8.99,0,0,0,0,0,3,97,1,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Barren Moor,Onslaught\n,C,,0,colorless,Barren Moor enters the battlefield tapped. : A...,0.29,0,0,0,0,0,1,95,1,NaT
Lightning Bolt,Modern Masters 2015\n,U,r,1,r,Lightning Bolt deals 3 damage to any target.,1.79,0,0,0,1,0,2,44,15,NaT
Wayfarer's Bauble,Commander 2015\n,C,1,1,colorless,", , Sacrifice Wayfarer's Bauble: Search your l...",1.29,0,0,0,0,0,1,148,6,NaT


### Dodawanie kolumny time since release

In [41]:
modern_df['time_since_release'] = pd.to_datetime('today') - modern_df['rel_date']
type(modern_df.time_since_release)

modern_df.time_since_release[0]

Timedelta('584 days 01:20:50.608963')

In [42]:
modern_df['time_since_release'] = modern_df['time_since_release'].apply(lambda x: get_days(x))
modern_df['time_since_release']

name
Ragavan, Nimble Pilferer        584.0
Sheoldred, the Apocalypse       136.0
Urza's Saga                     584.0
Fable of the Mirror-Breaker     339.0
Polluted Delta                 3041.0
                                ...  
Ethersworn Adjudicator           66.0
Astral Cornucopia              3272.0
Clockspinning                  5953.0
Gargos, Vicious Watcher        1291.0
Curse of the Swine              720.0
Name: time_since_release, Length: 8618, dtype: float64

In [43]:
modern_df['time_since_release'].value_counts() 

8423.0     790
1118.0     411
1046.0     355
66.0       303
640.0      211
          ... 
8668.0       3
10310.0      3
9367.0       2
8631.0       2
8379.0       1
Name: time_since_release, Length: 156, dtype: int64

In [44]:
mask = modern_df["time_since_release"].isna()
modern_df[mask]

,expansion,rarity,cost,cmc,color_identity,rules_text,price,is_w,is_u,is_b,is_r,is_g,rarity_num,rules_text_len,num_printings,rel_date,time_since_release
name,,,,,,,,,,,,,,,,,
"Sheoldred, the Apocalypse (Phyrexian)",Dominaria United Variants,M,2bb,4,b,"Deathtouch Whenever you draw a card, you gain ...",74.99,0,0,1,0,0,4,106,1,NaT,NaN
"Liliana, Dreadhorde General",War of the Spark\n,M,4bb,6,b,"Whenever a creature you control dies, draw a c...",27.99,0,0,1,0,0,4,247,1,NaT,NaN
"Rin and Seri, Inseparable (Buy-a-Box Foil)",Promotional\n,M,1rgw,4,rgw,"Whenever you cast a Dog spell, create a 1/1 gr...",20.99,1,0,0,1,1,4,292,1,NaT,NaN
"Vorinclex, Monstrous Raider (Phyrexian)",Kaldheim Variants,M,4gg,6,g,"Trample, haste If you would put one or more co...",49.99,0,0,0,0,1,4,352,1,NaT,NaN
Altar of the Brood,Khans of Tarkir\n,R,1,1,colorless,Whenever another permanent enters the battlefi...,8.99,0,0,0,0,0,3,97,1,NaT,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Diluvian Primordial,Game Night: Free-for-All,R,5uu,7,u,Flying When Diluvian Primordial enters the bat...,0.59,0,1,0,0,0,3,270,3,NaT,NaN
Geist of Saint Traft (Foil),Duel Decks: Blessed Vs. Cursed,M,1wu,3,uw,Hexproof (This creature can't be the target of...,0.99,1,1,0,0,0,4,257,1,NaT,NaN
Illicit Shipment,Streets of New Capenna\n,U,3bb,5,b,"Casualty 3 (As you cast this spell, you may sa...",0.25,0,0,1,0,0,2,189,1,NaT,NaN


### Usuwam NaNy

In [45]:
filename = 'modern_data_nans_PICKLE'
outfile = open(filename,'wb')
pickle.dump(modern_df,outfile)
outfile.close()

In [46]:
modern_df = pd.read_pickle('modern_data_nans_PICKLE')
modern_df.dropna(axis=0, subset=['time_since_release'], inplace=True)
modern_df.head()

,expansion,rarity,cost,cmc,color_identity,rules_text,price,is_w,is_u,is_b,is_r,is_g,rarity_num,rules_text_len,num_printings,rel_date,time_since_release
name,,,,,,,,,,,,,,,,,
"Ragavan, Nimble Pilferer",Modern Horizons 2,M,r,1,r,"Whenever Ragavan, Nimble Pilferer deals combat...",79.99,0,0,0,1,0,4,358,1,2021-06-18,584.0
"Sheoldred, the Apocalypse",Dominaria United,M,2bb,4,b,"Deathtouch Whenever you draw a card, you gain ...",79.99,0,0,1,0,0,4,106,1,2022-09-09,136.0
Urza's Saga,Modern Horizons 2,R,,0,colorless,"(As this Saga enters and after your draw step,...",42.99,0,0,0,0,0,3,381,1,2021-06-18,584.0
Fable of the Mirror-Breaker,Kamigawa: Neon Dynasty,R,2r,3,r,"(As this Saga enters and after your draw step,...",29.99,0,0,0,1,0,3,568,1,2022-02-18,339.0
Polluted Delta,Khans of Tarkir,R,,0,colorless,", Pay 1 life, Sacrifice Polluted Delta: Search...",44.99,0,0,0,0,0,3,136,3,2014-09-26,3041.0


In [47]:
modern_df.shape

(8176, 17)

In [48]:
filename = 'modern_df_ready_for_model_PICKLE'
outfile = open(filename,'wb')
pickle.dump(modern_df,outfile)
outfile.close()
